In [ ]:
## read twitter_handle.csv
## if sentiment-twitter handle exist then im done
## if it doesn't exist read twitter_handle.csv
## sentiment-@twitter_handle.csv

##receiving a timestamp, username, followers, text
## add new column to sentiment score
## resample into 15 minute buckets
## average them (weighted average based on sentiment score and followers)
## VWOP
### weighted score = followers * sentiment score + followers * sentiment score / total number of followers

## output file = time stamp, weigted sentiment score, total number of tweets, total number of followers

In [81]:
import numpy as np
import pandas as pd
from datetime import datetime
import hvplot as hv
import spacy
import asent
from dframcy import dframcy
from spacytextblob.spacytextblob import SpacyTextBlob


In [68]:
#Create Pandas DataFrame called sscore_df for sentiment score dataframe.

sscore_df = pd.DataFrame()

tweet_df = pd.read_csv("../Files/demo_tweets.csv", index_col="date", infer_datetime_format=True, parse_dates=True)

tweet_df.dropna(inplace=True)

tweet_df

,username,followers,tweet
date,,,
2021-09-25,YLQI,7857.0,Is history repeating itself...?#DONTNORMALIZEH...
2021-09-26,HTOQ,4586.0,@barackobama Thank you for your incredible gra...
2021-09-27,WFWO,1621.0,Life goals. https://t.co/XIn1qKMKQl
2021-09-28,PLVO,7050.0,Me right now 🙏🏻 https://t.co/gW55C1wrwd
2021-09-29,QVCD,1141.0,SISTERS ARE DOIN' IT FOR THEMSELVES! 🙌🏻💪🏻❤️ ht...
...,...,...,...
2022-09-19,CWMM,7763.0,@katyscrush hola bb
2022-09-20,QHFE,3946.0,@waIkingonair yes this
2022-09-21,JMLA,1194.0,@katycatsophia @camilasviews JK ily


In [70]:
nlp = spacy.blank('en')
nlp.add_pipe('sentencizer')
nlp.add_pipe('asent_en_v1')
nlp
tweet_df["Sentiment_asent"] = tweet_df["tweet"].apply(lambda tweet: nlp(tweet)._.polarity.compound)

tweet_df.head(5)

,username,followers,tweet,Sentiment_1,Sentiment_asent
date,,,,,
2021-09-25,YLQI,7857.0,Is history repeating itself...?#DONTNORMALIZEH...,0.000000,0.000000
2021-09-26,HTOQ,4586.0,@barackobama Thank you for your incredible gra...,0.680111,0.680111
2021-09-27,WFWO,1621.0,Life goals. https://t.co/XIn1qKMKQl,0.000000,0.000000
2021-09-28,PLVO,7050.0,Me right now 🙏🏻 https://t.co/gW55C1wrwd,0.000000,0.000000
2021-09-29,QVCD,1141.0,SISTERS ARE DOIN' IT FOR THEMSELVES! 🙌🏻💪🏻❤️ ht...,0.353378,0.353378


In [74]:
nlp2 = spacy.load('en_core_web_sm')
nlp2.add_pipe('spacytextblob')
nlp2
tweet_df['Sentiment_textblob'] = tweet_df['tweet'].apply(lambda tweet: nlp2(tweet)._.blob.polarity)



#tweet_df.drop(columns="Sentiment_1", inplace=True)

tweet_df.head(5)

,username,followers,tweet,Sentiment_asent,Sentiment_textblob
date,,,,,
2021-09-25,YLQI,7857.0,Is history repeating itself...?#DONTNORMALIZEH...,0.000000,0.000000
2021-09-26,HTOQ,4586.0,@barackobama Thank you for your incredible gra...,0.680111,0.900000
2021-09-27,WFWO,1621.0,Life goals. https://t.co/XIn1qKMKQl,0.000000,0.000000
2021-09-28,PLVO,7050.0,Me right now 🙏🏻 https://t.co/gW55C1wrwd,0.000000,0.285714
2021-09-29,QVCD,1141.0,SISTERS ARE DOIN' IT FOR THEMSELVES! 🙌🏻💪🏻❤️ ht...,0.353378,0.000000


In [79]:
tweet_df['average_sentiment'] = tweet_df[["Sentiment_asent", "Sentiment_textblob"]].mean(axis=1)

In [80]:
tweet_df.head(5)

,username,followers,tweet,Sentiment_asent,Sentiment_textblob,average_sentiment
date,,,,,,
2021-09-25,YLQI,7857.0,Is history repeating itself...?#DONTNORMALIZEH...,0.000000,0.000000,0.000000
2021-09-26,HTOQ,4586.0,@barackobama Thank you for your incredible gra...,0.680111,0.900000,0.790056
2021-09-27,WFWO,1621.0,Life goals. https://t.co/XIn1qKMKQl,0.000000,0.000000,0.000000
2021-09-28,PLVO,7050.0,Me right now 🙏🏻 https://t.co/gW55C1wrwd,0.000000,0.285714,0.142857
2021-09-29,QVCD,1141.0,SISTERS ARE DOIN' IT FOR THEMSELVES! 🙌🏻💪🏻❤️ ht...,0.353378,0.000000,0.176689


In [84]:
### weighted score = followers * sentiment score + followers * sentiment score / total number of followers

tweet_df['weighted_sentiment'] = (tweet_df["followers"])*(tweet_df["average_sentiment"]).sum()/tweet_df["followers"].sum()
 
tweet_df

,username,followers,tweet,Sentiment_asent,Sentiment_textblob,average_sentiment,weighted_sentiment
date,,,,,,,
2021-09-25,YLQI,7857.0,Is history repeating itself...?#DONTNORMALIZEH...,0.000000,0.000000,0.000000,0.191978
2021-09-26,HTOQ,4586.0,@barackobama Thank you for your incredible gra...,0.680111,0.900000,0.790056,0.112055
2021-09-27,WFWO,1621.0,Life goals. https://t.co/XIn1qKMKQl,0.000000,0.000000,0.000000,0.039608
2021-09-28,PLVO,7050.0,Me right now 🙏🏻 https://t.co/gW55C1wrwd,0.000000,0.285714,0.142857,0.172260
2021-09-29,QVCD,1141.0,SISTERS ARE DOIN' IT FOR THEMSELVES! 🙌🏻💪🏻❤️ ht...,0.353378,0.000000,0.176689,0.027879
...,...,...,...,...,...,...,...
2022-09-19,CWMM,7763.0,@katyscrush hola bb,0.000000,0.000000,0.000000,0.189682
2022-09-20,QHFE,3946.0,@waIkingonair yes this,0.401924,0.000000,0.200962,0.096417
2022-09-21,JMLA,1194.0,@katycatsophia @camilasviews JK ily,0.792514,0.000000,0.396257,0.029174


In [60]:
tweet, polarity = tweet_text2[20]
print(polarity)
asent.visualize(nlp(tweet), style='prediction')
asent.visualize(nlp(tweet), style='analysis')

0.4444444444444445


In [ ]:

rand_sent = np.random.randint(-1, 1, 365, int)
rand_sent.tolist()
sscore_df["score"] = rand_sent 

date = pd.date_range(start='09/25/2021',end='09/25/2022',periods=365)

In [ ]:
tweet_df['tweet']

In [ ]:
sscore_df["date"] = date

In [ ]:
sscore_df.reset_index()
sscore_df.set_index("date")
sscore_df